# Scala school advanced types

- View bounds ("type classes")
- Other Type Bounds
- Higher kinded types & ad-hoc polymorphism
- F-bounded polymophism / recursive types
- Structural types
- Abstract types members
- Type erasures & manifests
- Case study: Finagle

# View bounds ("type classes")

In [1]:
implicit def strToInt(x: String) = x.toInt

defined function strToInt

In [2]:
"123"

res1: String = "123"

In [3]:
val y: Int = "123"

y: Int = 123

In [4]:
math.max("123", 111)

res3: Int = 123

In [5]:
class Container[A <% Int] { 
    def addIt(x: A) = 123 + x
}

defined class Container

In [7]:
(new Container[String]).addIt("123")

res5: Int = 246

In [10]:
(new Container[Int]).addIt(123)

res7: Int = 246

In [9]:
(new Container[Float]).addIt(123.2F)

: 

# Other type bounds

Numeric[T]가 정의되어 있음

<:<, <%<는 2.10 이후에는 없어짐.

In [13]:
class Container[A](value: A) {
    def addIt(implicit evidence: A =:= Int) = 123 + value
}

defined class Container

In [14]:
(new Container(123)).addIt

res9: Int = 246

In [15]:
(new Container("123")).addIt

: 

In [16]:
class Container[A](value: A) {
    def addIt(implicit evidence: A <%< Int) = 123 + value
}

: 

In [17]:
(new Container("123")).addIt

: 

## Generic programming with views

In [18]:
List(1,2,3,4).min

res10: Int = 1

In [19]:
List(1,2,3,4).min(new Ordering[Int] {def compare(a: Int, b: Int) = b compare a})

res11: Int = 4

In [22]:
trait LowPriorityOrderingImplicits {
    implicit def ordered[A <: Ordered[A]]: Ordering[A] = new Ordering[A] {
        def compare(x: A, y: A) = x.compare(y)
    }
}

defined trait LowPriorityOrderingImplicits

## Context bounds & implicitly[]

In [3]:
def foo[A](implicit x: Ordered[A]) {}

defined function foo

In [4]:
def foo[A: Ordered] {}

defined function foo

In [5]:
implicitly[Ordering[Int]]

res4: Ordering[Int] = scala.math.Ordering$Int$@29a39fe5

# Higher kinded types & ad-hoc polymorphism 

In [6]:
trait Container[M[_]] { def put[A](x: A): M[A]; def get[A](m: M[A]): A }

defined trait Container

In [8]:
val container = new Container[List] { def put[A](x: A) = List(x); def get[A](m: List[A]) = m.head }

container: AnyRef with Container[List] = cmd7$$user$$anonfun$1$$anon$1@3c8d785e

In [9]:
container.put("hey")
container.put(123)

res8_0: List[String] = List("hey")
res8_1: List[Int] = List(123)

In [17]:
trait Container[M[_]] { def put[A](x: A): M[A]; def get[A](m: M[A]): A }

defined trait Container

In [18]:
implicit val listContainer = new Container[List] { def put[A](x: A) = List(x); def get[A](m: List[A]) = m.head }

listContainer: AnyRef with Container[List] = cmd16$$user$$anonfun$1$$anon$1@20d517d6

In [19]:
implicit val optionContainer = new Container[Some] { def put[A](x: A) = Some(x); def get[A](m: Some[A]) = m.get }

optionContainer: AnyRef with Container[Some] = cmd17$$user$$anonfun$1$$anon$1@5870c8d2

In [20]:
def tupleize[M[_]: Container, A, B](fst: M[A], snd: M[B]) = {
    val c = implicitly[Container[M]]
    c.put(c.get(fst), c.get(snd))
}

defined function tupleize

In [21]:
tupleize(Some(1), Some(2))

res19: Some[(Int, Int)] = Some((1, 2))

In [22]:
tupleize(List(1), List(2))

res20: List[(Int, Int)] = List((1, 2))

# F-bounded polymophism / recursive types

In [28]:
trait Container extends Ordered[Container]

defined trait Container

In [29]:
class MyContainer extends Container {
  def compare(that: MyContainer): Int = 0
}

: 

In [30]:
trait Container[A <: Container[A]] extends Ordered[A]

defined trait Container

In [31]:
class MyContainer extends Container[MyContainer] { 
  def compare(that: MyContainer) = 0 
}

defined class MyContainer

In [33]:
List(new MyContainer, new MyContainer, new MyContainer)
List(new MyContainer, new MyContainer, new MyContainer).min

res25_0: List[MyContainer] = List(
  cmd23$$user$MyContainer@1ae62981,
  cmd23$$user$MyContainer@7caee1f2,
  cmd23$$user$MyContainer@3d7416d1
)
res25_1: MyContainer = cmd23$$user$MyContainer@7900ee0a

In [37]:
class YourContainer extends Container[YourContainer] { def compare(that: YourContainer) = 0 }

defined class YourContainer

In [38]:
List(new MyContainer, new MyContainer, new MyContainer, new YourContainer)   

: 

In [39]:
(new MyContainer, new MyContainer, new MyContainer, new YourContainer).min

: 

https://en.wikipedia.org/wiki/Bounded_quantification

http://www.slideshare.net/b0ris_1/so-different-polymorphism-in-scala

# Structural types

In [21]:
def foo(x: { def get: Int }) = 123 + x.get

defined function foo

In [22]:
foo(new { def get = 10 })            

res12: Int = 133

# Abstract types members

In [40]:
trait Foo { type A; val x: A; def getX: A = x }

(new Foo { type A = Int; val x = 123 }).getX   

(new Foo { type A = String; val x = "hey" }).getX

defined trait Foo
res28_1: Int = 123
res28_2: String = "hey"

In [41]:
trait Foo[M[_]] { type t[A] = M[A] }

val x: Foo[List]#t[Int] = List(1)

defined trait Foo
x: List[Int] = List(1)

# Type erasures & manifests

In [26]:
class MakeFoo[A](implicit manifest: Manifest[A]) {
    def make: A = manifest.erasure.newInstance.asInstanceOf[A]
}
(new MakeFoo[String]).make

defined class MakeFoo
res18_1: String = ""

http://www.scala-lang.org/api/2.11.6/index.html#scala.reflect.Manifest

http://docs.scala-lang.org/overviews/reflection/typetags-manifests.html

# Case study: Finagle